# Designing a Buck Inductor with PyOpenMagnetics

This notebook walks through the complete process of designing an inductor for a buck converter.

## Design Specifications

- **Input voltage**: 48V
- **Output voltage**: 12V
- **Output current**: 10A
- **Switching frequency**: 100 kHz
- **Maximum ripple**: 20% of output current

In [ ]:
import PyOpenMagnetics
import json
from pprint import pprint

# Design parameters
V_IN = 48       # Input voltage (V)
V_OUT = 12      # Output voltage (V)
I_OUT = 10      # Output current (A)
F_SW = 100000   # Switching frequency (Hz)
RIPPLE = 0.20   # Maximum current ripple (20%)

# Calculate required inductance
duty_cycle = V_OUT / V_IN
delta_I = RIPPLE * I_OUT  # Peak-to-peak ripple current
L_min = (V_IN - V_OUT) * duty_cycle / (F_SW * delta_I)

print(f"Duty Cycle: {duty_cycle:.2%}")
print(f"Ripple Current: {delta_I:.2f} A peak-to-peak")
print(f"Minimum Inductance: {L_min * 1e6:.1f} µH")

## Step 1: Define Design Requirements

In [ ]:
# Create design requirements for the MAS schema
design_requirements = {
    "name": "Buck 48V to 12V Inductor",
    "magnetizingInductance": {
        "nominal": L_min * 1.2  # Add 20% margin
    },
    "turnsRatios": []  # Empty for single-winding inductor
}

print(f"Target inductance: {design_requirements['magnetizingInductance']['nominal'] * 1e6:.1f} µH")

## Step 2: Define Operating Conditions

In [ ]:
# Define the operating point with current waveform
I_PEAK = I_OUT + delta_I / 2
I_VALLEY = I_OUT - delta_I / 2

operating_point = {
    "name": "Nominal Load",
    "conditions": {
        "ambientTemperature": 40
    },
    "excitationsPerWinding": [
        {
            "name": "Primary",
            "frequency": F_SW,
            "current": {
                "processed": {
                    "label": "Triangular",
                    "dutyCycle": duty_cycle,
                    "peakToPeak": delta_I,
                    "offset": I_OUT
                }
            },
            "voltage": {
                "processed": {
                    "label": "Rectangular",
                    "dutyCycle": duty_cycle,
                    "peakToPeak": V_IN,
                    "offset": -V_OUT
                }
            }
        }
    ]
}

print(f"Current: {I_VALLEY:.2f}A to {I_PEAK:.2f}A")
print(f"Frequency: {F_SW / 1000:.0f} kHz")

## Step 3: Use the Core Adviser

In [ ]:
# Complete inputs structure
inputs = {
    "designRequirements": design_requirements,
    "operatingPoints": [operating_point]
}

# Process inputs first (required before adviser)
processed = PyOpenMagnetics.process_inputs(inputs)
print("Inputs processed successfully!")

# Get magnetic design recommendations
print("\nSearching for optimal designs...")
try:
    magnetics = PyOpenMagnetics.calculate_advised_magnetics(
        processed,
        max_results=5,
        core_mode="STANDARD_CORES"
    )
    
    print(f"\nFound {len(magnetics)} suitable designs:")
    for i, result in enumerate(magnetics[:5]):
        if 'magnetic' in result:
            magnetic = result['magnetic']
            core = magnetic['core']['functionalDescription']
            shape_name = core['shape']['name'] if isinstance(core['shape'], dict) else core['shape']
            mat_name = core['material']['name'] if isinstance(core['material'], dict) else core['material']
            print(f"  {i+1}. {shape_name} / {mat_name}")
except Exception as e:
    print(f"Adviser error (may need more compute time): {e}")

## Step 4: Design the Winding

In [ ]:
# Create a manual core for winding analysis
core_data = {
    "functionalDescription": {
        "name": "Buck Inductor Core",
        "type": "two-piece set",
        "shape": "ETD 39/20/13",
        "material": "3C95",
        "gapping": [
            {"type": "subtractive", "length": 0.001}  # 1mm gap
        ],
        "numberStacks": 1
    }
}

# Calculate core processed data
core = PyOpenMagnetics.calculate_core_data(core_data, False)
print(f"Core: {core['functionalDescription']['shape']}")

# Create a basic bobbin
bobbin = PyOpenMagnetics.create_basic_bobbin(core, 0.001)  # 1mm margin
print("Bobbin created for winding")

# Get wire for winding
wires = PyOpenMagnetics.get_wire_names()
round_wires = [w for w in wires if "Round" in w and "1." in w][:5]
print(f"Available round wires: {round_wires}")

## Step 5: Analyze Using the Complete Adviser Results

If the adviser returned results, analyze them.

In [ ]:
# Analyze adviser results if available
try:
    if 'magnetics' in dir() and len(magnetics) > 0:
        print("=== Best Design Analysis ===\n")
        
        best = magnetics[0]
        if 'magnetic' in best:
            magnetic = best['magnetic']
            core_fd = magnetic['core']['functionalDescription']
            
            # Extract shape and material names
            shape = core_fd['shape']['name'] if isinstance(core_fd['shape'], dict) else core_fd['shape']
            material = core_fd['material']['name'] if isinstance(core_fd['material'], dict) else core_fd['material']
            
            print(f"Shape: {shape}")
            print(f"Material: {material}")
            
            # Check gapping
            gapping = core_fd.get('gapping', [])
            total_gap = sum(g.get('length', 0) for g in gapping) * 1000
            print(f"Total gap: {total_gap:.2f} mm")
            
            # Get coil info
            if 'coil' in magnetic:
                coil = magnetic['coil']
                if 'functionalDescription' in coil:
                    for winding in coil['functionalDescription']:
                        print(f"Winding: {winding.get('name', 'Primary')}")
                        print(f"  Turns: {winding.get('numberTurns', 'N/A')}")
                        print(f"  Parallels: {winding.get('numberParallels', 1)}")
            
            # Show outputs if available
            if 'outputs' in best:
                outputs = best['outputs']
                print(f"\nSimulation Results:")
                if 'coreLosses' in outputs:
                    print(f"  Core losses: {outputs['coreLosses']:.3f} W")
                if 'windingLosses' in outputs:
                    print(f"  Winding losses: {outputs['windingLosses']:.3f} W")
    else:
        print("No adviser results available. Run the adviser cell first.")
except Exception as e:
    print(f"Analysis error: {e}")

## Step 6: Visualize the Design

In [ ]:
try:
    import matplotlib.pyplot as plt
    import numpy as np
    
    # Visualize current waveform
    t = np.linspace(0, 2/F_SW, 500)  # Two periods
    
    # Generate triangular waveform
    T = 1 / F_SW
    t_on = duty_cycle * T
    
    current = np.zeros_like(t)
    for i, ti in enumerate(t):
        t_mod = ti % T
        if t_mod < t_on:
            # Rising slope during on-time
            current[i] = I_VALLEY + (I_PEAK - I_VALLEY) * (t_mod / t_on)
        else:
            # Falling slope during off-time
            current[i] = I_PEAK - (I_PEAK - I_VALLEY) * ((t_mod - t_on) / (T - t_on))
    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))
    
    # Current waveform
    ax1.plot(t * 1e6, current, 'b-', linewidth=2)
    ax1.axhline(y=I_OUT, color='r', linestyle='--', label=f'DC = {I_OUT}A')
    ax1.fill_between(t * 1e6, current, I_OUT, alpha=0.3)
    ax1.set_xlabel('Time (µs)', fontsize=12)
    ax1.set_ylabel('Inductor Current (A)', fontsize=12)
    ax1.set_title('Buck Inductor Current Waveform', fontsize=14)
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.set_xlim(0, t[-1] * 1e6)
    
    # Loss breakdown pie chart
    if 'outputs' in dir() and 'outputs' in results:
        core_loss = outputs.get('coreLosses', 0)
        winding_loss = outputs.get('windingLosses', 0)
        if core_loss > 0 or winding_loss > 0:
            ax2.pie(
                [core_loss, winding_loss],
                labels=['Core Losses', 'Winding Losses'],
                autopct='%1.1f%%',
                colors=['#ff6b6b', '#4ecdc4'],
                explode=[0.02, 0.02],
                startangle=90
            )
            ax2.set_title('Loss Distribution', fontsize=14)
        else:
            ax2.text(0.5, 0.5, 'No loss data available', ha='center', va='center')
            ax2.set_xlim(0, 1)
            ax2.set_ylim(0, 1)
    else:
        ax2.text(0.5, 0.5, 'Run simulation first', ha='center', va='center')
        ax2.set_xlim(0, 1)
        ax2.set_ylim(0, 1)
    
    plt.tight_layout()
    plt.show()
    
except ImportError:
    print("Install matplotlib for visualizations: pip install matplotlib")

## Summary

This notebook demonstrated:

1. **Calculating inductance** from buck converter specifications
2. **Using the core adviser** to find suitable cores
3. **Designing windings** for the selected core
4. **Simulating losses** with physics-based models
5. **Visualizing** current waveforms and loss distribution

## Exercises

1. Try different switching frequencies and observe the impact on losses
2. Experiment with different wire types (litz, foil)
3. Compare multiple core shapes for the same requirements
4. Add a gapped core to achieve the target inductance